# Ensembles:

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier

In [2]:
# Load csv to DataFrame:
file_path = Path('Resources/LoanStats_2019Q1.csv')
stats_df = pd.read_csv(file_path)
stats_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [3]:
# Create LabelEncoder instance:
le = LabelEncoder()

# Encode home_ownership column:
le.fit(stats_df['home_ownership'])
stats_df['home_ownership'] = le.transform(stats_df['home_ownership'])

# Encode verification_status column:
le.fit(stats_df['verification_status'])
stats_df['verification_status'] = le.transform(stats_df['verification_status'])

# Encode issue_d column:
le.fit(stats_df['issue_d'])
stats_df['issue_d'] = le.transform(stats_df['issue_d'])

# Encode pymnt_plan column:
le.fit(stats_df["pymnt_plan"])
stats_df['pymnt_plan'] = le.transform(stats_df['pymnt_plan'])

# Encode loan_status column:
le.fit(stats_df['loan_status'])
stats_df['loan_status'] = le.transform(stats_df['loan_status'])

# Encode next_pymnt_d column:
le.fit(stats_df['next_pymnt_d'])
stats_df['next_pymnt_d'] = le.transform(stats_df['next_pymnt_d'])

# Encode application_type column:
le.fit(stats_df['application_type'])
stats_df['application_type'] = le.transform(stats_df['application_type'])

# Encode initial_list_status column:
le.fit(stats_df['initial_list_status'])
stats_df['initial_list_status'] = le.transform(stats_df['initial_list_status'])

# Encode hardship_flag column:
le.fit(stats_df['hardship_flag'])
stats_df['hardship_flag'] = le.transform(stats_df['hardship_flag'])

# Encode debt_settlement_flag column:
le.fit(stats_df['debt_settlement_flag'])
stats_df['debt_settlement_flag'] = le.transform(stats_df['debt_settlement_flag'])

stats_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,2,1,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,2,1,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,2,1,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,2,1,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,2,1,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


## Split the Data into Training and Testing

In [4]:
# Define target:
y = stats_df['loan_status']

# Check the balance of target values:
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [5]:
# Define features:
x = stats_df.copy()
x.drop('loan_status',axis=1,inplace=True)
x.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [6]:
# Split x and y into x_train, x_test, y_train, y_test:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=78)

## Data Pre-Processing

Scale the training and testing data using `StandardScaler`.

In [7]:
# Create a SandardScaler instance:
scaler = StandardScaler()

# Fit the Standard Scaler:
x_scaler = scaler.fit(x_train)

# Scale training and testing data:
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

## Ensemble Learners

Compare the BalancedRandomForestClassifier and EasyEnsembleClassifier algorithms and determine which results in the best performance then complete the folliowing for each algorithm:

1. Train the model using the training data. 
2. Calculate the Balanced Accuracy Score from sklearn.metrics.
3. Display the Confusion Matrix from sklearn.metrics.
4. Generate an Imbalanced Classification Report from imbalanced-learn.
5. For the Balanced Random Forest Classifier, output the feature importance in descending order (most important to least important) along with the feature score.

### Balanced Random Forest Classifier

In [8]:
# Resample the training data with the BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100,random_state=1)
brf_model = brf_model.fit(x_train_scaled,y_train)

In [9]:
# Calculate the balanced accuracy score:
brf_predictions = brf_model.predict(x_test_scaled)
brf_ba_score = balanced_accuracy_score(y_test,brf_predictions)
display(brf_ba_score)

0.7518737344353009

In [10]:
# Display the confusion matrix:
brf_cm = confusion_matrix(y_test,brf_predictions)
brf_cm_df = pd.DataFrame(brf_cm,index=['Actual 0','Actual 1'],columns=['Predicted 0','Predicted 1'])
brf_cm_df

,Predicted 0,Predicted 1
Actual 0,58,34
Actual 1,2168,14945


In [11]:
# Print the imbalanced classification report:
print(classification_report_imbalanced(y_test,brf_predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.63      0.87      0.05      0.74      0.54        92
          1       1.00      0.87      0.63      0.93      0.74      0.56     17113

avg / total       0.99      0.87      0.63      0.93      0.74      0.56     17205



In [12]:
# List features in descending order by importance:
importance_sorted = sorted(zip(brf_model.feature_importances_,x.columns),reverse=True)
display(importance_sorted)

[(0.07972578916270898, 'total_rec_prncp'),
 (0.07184199283761979, 'last_pymnt_amnt'),
 (0.06428200815827503, 'total_pymnt'),
 (0.05361500880138094, 'total_pymnt_inv'),
 (0.04397653617040098, 'total_rec_int'),
 (0.03372491650210961, 'int_rate'),
 (0.0261889114749861, 'issue_d'),
 (0.019458020914104493, 'mths_since_recent_inq'),
 (0.019412889680972666, 'mo_sin_old_rev_tl_op'),
 (0.017719802114217368, 'dti'),
 (0.016511806392147194, 'bc_open_to_buy'),
 (0.016230999521916263, 'out_prncp'),
 (0.015727577793596847, 'annual_inc'),
 (0.015518209726778413, 'revol_bal'),
 (0.01537638021631967, 'installment'),
 (0.01534576834053173, 'il_util'),
 (0.01524337492856352, 'bc_util'),
 (0.014804822806011017, 'all_util'),
 (0.014168797191308665, 'mo_sin_old_il_acct'),
 (0.013807745211418582, 'total_bal_ex_mort'),
 (0.013706135412067779, 'max_bal_bc'),
 (0.013600058476436798, 'out_prncp_inv'),
 (0.013370828968501867, 'total_bc_limit'),
 (0.013291160492260465, 'tot_cur_bal'),
 (0.013167436261829158, 'num_

### Easy Ensemble Classifier

In [13]:
# Train the EasyEnsembleClassifier:
eec_model = EasyEnsembleClassifier(n_estimators=100,random_state=1)
eec_model.fit(x_train_scaled,y_train)
y_pred_eec = eec_model.predict(x_test)

In [14]:
# Calculate balanced accuracy score:
eec_ba_score = balanced_accuracy_score(y_test,y_pred_eec)
display(eec_ba_score)

0.48017112594290123

In [15]:
# Display confusion matrix:
eec_cm = confusion_matrix(y_test,y_pred_eec)
eec_cm_df = pd.DataFrame(eec_cm,index=['Actual 0','Actual 1'],columns=['Predicted 0','Predicted 1'])
eec_cm_df

,Predicted 0,Predicted 1
Actual 0,31,61
Actual 1,6445,10668


In [16]:
# Print imbalanced classification report:
print(classification_report_imbalanced(y_test,y_pred_eec))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.34      0.62      0.01      0.46      0.20        92
          1       0.99      0.62      0.34      0.77      0.46      0.22     17113

avg / total       0.99      0.62      0.34      0.76      0.46      0.22     17205



# Final Questions and Analysis:

### __1. Which model had the best balanced accuracy score?__
    * The Balanced Random Forest Classifier had the best Balanced Accuracy Score. 

### __2. Which model had the best recall score?__
    * The Balanced Random Forest Classifier had the best Recall Score.

_The recall scores of each model are output below:_

In [17]:
print(f'BalancedRandomForestClassifier Recall Score = {recall_score(y_test,brf_predictions)}\nEasyEnsembleClassifier Recall Score = {recall_score(y_test,y_pred_eec)}')

BalancedRandomForestClassifier Recall Score = 0.8733126862619062
EasyEnsembleClassifier Recall Score = 0.6233857301466721


### __3. Which model had the best geometric mean score?__
    * The Balanced Random Forest Classifier had the best Geometric Mean Score.

_The geometric mean scores of each model are output below:_

In [18]:
print(f'BalancedRandomForestClassifier Geometric Mean Score = {geometric_mean_score(y_test,brf_predictions)}\nEasyEnsembleClassifier Geometric Mean Score = {geometric_mean_score(y_test,y_pred_eec)}')

BalancedRandomForestClassifier Geometric Mean Score = 0.7420018150334545
EasyEnsembleClassifier Geometric Mean Score = 0.4583163616237488


### __4. What are the top three features?__
    * The top 3 features are:
        * total_rec_prncp
        * last_pymnt_amnt
        * total_pymnt

_The top 3 features are output below:_

In [19]:
importance_sorted = sorted(zip(brf_model.feature_importances_,x.columns),reverse=True)
importance_sorted[:3]

[(0.07972578916270898, 'total_rec_prncp'),
 (0.07184199283761979, 'last_pymnt_amnt'),
 (0.06428200815827503, 'total_pymnt')]